In [39]:
'''
SEIRS-CoV-2 SEIRS Model Proof-of-Concept Notebook
Author: Eric Renart
Last Update: 08/15/2021
'''

import numpy as np
import pandas as pd
import seaborn as sbn
from scipy.integrate import solve_ivp
from matplotlib import pyplot as pp
from sklearn.linear_model import LinearRegression

In [40]:
def ode_function(Y, rates, probabilities, t):
    S, E, A, I, H, ICU, VENT, D, R, VAX = Y
    beta, gamma, gamma_H, gamma_ICU, gamma_VENT, delta, delta_H, delta_ICU, delta_VENT, delta_D, phi, omega, lambda_mu = rates
    P_asymptomatic, P_H_from_I, P_ICU_from_H, P_VENT_from_ICU, P_D_from_VENT, P_INIT_VAX = probabilities
    N = S + E + A + I + H + ICU + VENT + D + R + VAX

    # Calculate inverse probabilities
    P_R_from_I = 1 - P_H_from_I
    P_R_from_H = 1 - P_ICU_from_H
    P_R_from_ICU = 1 - P_VENT_from_ICU
    P_R_from_VENT = 1 - P_D_from_VENT
    P_symptomatic = 1 - P_asymptomatic

    '''
    Differential ODE's
    Assumptions:
    1. Birth and Death rates are equal (LAMBDA = mu)
    2. Everyone is ventilated in ICU before dying (ie. no excess deaths apart from natural deathrate)
    3. People can only get vaccinated when susceptible or recovered
    4. Probabilities do not depend on age (Age-dependence TBI)
    5. Vaccines are 100% effective (effects of VOCs TBI)
    '''

    S_over_N = S / N
    total_I = I + A # Symptomatics and asymptomatics are both infectious
    dSdt = lambda_mu - (lambda_mu * S) - (beta * total_I * S_over_N) - (phi * P_INIT_VAX * S) + (omega * R)
    dEdt = (beta * total_I * S_over_N) - delta * (P_asymptomatic * A - P_symptomatic * I)
    dAdt = (delta * P_asymptomatic * E) - (gamma * A)
    dIdt = (delta * P_symptomatic * E) - (delta_H * P_H_from_I * I) - (gamma * P_R_from_I * I)
    dHdt = (delta_H * P_H_from_I * I) - (delta_ICU * P_ICU_from_H * H) - (gamma_H * P_R_from_H * H)
    dICUdt = (delta_ICU * P_ICU_from_H * H) - (delta_VENT * P_VENT_from_ICU * ICU) - (gamma_ICU * P_R_from_ICU * ICU)
    dVENTdt = (delta_VENT * P_VENT_from_ICU * ICU) - (delta_D * P_D_from_VENT * VENT) - (gamma_VENT * P_R_from_VENT * VENT)
    dDdt = deltaD * P_D_from_VENT * VENT
    dRdt = (gamma * total_I) + (gamma_H * H) + (gamma_ICU * ICU) + (gamma_VENT * VENT) - (omega * R) - (phi * P_INIT_VAX * R)
    dVAXdt = phi * P_INIT_VAX * (S + R)

    return (dSdt, dEdt, dAdt, dIdt, dHdt, dICUdt, dVENTdt, dDdt, dRdt, dVAXdt)


In [41]:
def run(initials, probabilities, rates, length):
    t = np.linspace(0,length,length)
    results_ts = solve_ivp(ode_function, initials, t, args=(probabilities, rates))
    results_df = pd.DataFrame(results_ts, dtype='int32', columns=['Day', 'Susceptible', 'Exposed', 'Asymptomatic', 'Symptomatic', 'Hospitalized', 'In ICU', 'On Ventilator', 'Dead', 'Recovered', 'Vaccinated'])
    results_df['Total Check'] = results_df.sum(axis=1)
    results_df['Active Cases'] = results_df['Exposed'] + results_df['Asymptomatic'] + results_df['Symptomatic'] + results_df['Hospitalized'] + results_df['In ICU'] + results_df['On Ventilator']
    return results_df

In [42]:
# Virus parameters

# R0. TODO: use current R0 from CAN
R0_base = 1.3

# Natural Birth+Death Rate per day
birth_death_rate = 100

# Times
t_incubation = 5 # Incubation Period from infection to symptoms if they appear
t_symptomatic_to_hospitalized = 7
t_hospitalized_to_icu = 7
t_icu_to_ventilator = 2
t_ventilator_to_dead = 7
t_mild_asymptomatic_to_recovered = 7
t_hospitalized_to_recovered = 10
t_icu_to_recovered = 15
t_ventilated_to_recovered = 17
t_immunity_loss = 360
t_vaccine_immunity = 56

# Probabilities
p_asymptomatic = 0.35
p_symptomatic_to_hospitalized = 0.15
p_hospitalized_to_icu = 0.20
p_icu_to_ventilator = 0.55
p_ventilator_to_dead = 0.80
vaccine_hesitance = 0.25

# Construct the inputs to model
gamma = 1 / t_mild_asymptomatic_to_recovered
beta = R0_base * gamma

# Calculate rates of state transitions
rates = [beta, gamma, 1/t_hospitalized_to_recovered, 1/t_icu_to_recovered, 1/t_ventilated_to_recovered, 1/t_incubation, 1/t_symptomatic_to_hospitalized, 1/t_hospitalized_to_icu, 1/t_icu_to_ventilator, 1/t_ventilator_to_dead, 1/t_vaccine_immunity,
1/t_immunity_loss, 1/birth_death_rate]

# Probabilities of state transitions
probabilities = [p_asymptomatic, p_symptomatic_to_hospitalized, p_hospitalized_to_icu, p_icu_to_ventilator, p_ventilator_to_dead, 1-vaccine_hesitance]

In [43]:
# Initial populatiion conditions
S0 = 999999
E0 = 1
A0 = 0
I0 = 0
H0 = 0
ICU0 = 0
VENT0 = 0
D0 = 0
R0 = 0
VAX0 = 0
initials = [S0, E0, A0, I0, H0, ICU0, VENT0, D0, R0, VAX0]

In [45]:
# Run model
sim_length = 540
results = run(initials, rates, probabilities, sim_length)
results

TypeError: cannot unpack non-iterable float object